# SVI Part II: Conditional Independence, Subsampling, and Amortization

**This Tutorial is adapted from [https://pyro.ai/examples/svi_part_ii.html](https://pyro.ai/examples/svi_part_ii.html)*

## The Goal: Scaling SVI to Large Datasets

For a model with $N$ observations, running the **model** and **guide** and constructing the ELBO involves evaluating log pdf's whose complexity scales badly with $N$. This is a problem if we want to scale to large datasets. Luckily, the ELBO objective naturally supports subsampling provided that our model/guide have some conditional independence structure that we can take advantage of. For example, in the case that the observations are conditionally independent given the latents, the log likelihood term in the ELBO can be approximated with

$\sum_{i=1}^N \log p({\bf x}_i | {\bf z}) \approx  \frac{N}{M} \sum_{i\in{\mathcal{I}_M}} \log p({\bf x}_i | {\bf z})$

where $\mathcal{I}_M$ is a mini-batch of indices of size $M$ with $M < N$. So how do we do this in Pyro?

## Marking Conditional Independence in Pyro

If we want to do this sort of thing in Pyro, we first need to make sure that the **model** and **guide** are written in such a way that Pyro can leverage the relevant conditional independencies. Let's see how this is done. Pyro provides two language primitives for marking conditional independencies: plate and markov. Let's start with the simpler of the two.

### Sequential [plate]

Let's return to the example we used in the previous tutorial. For convenience let's replicate the main logic of model here:

In [1]:
def model(data):
    
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    
    # loop over the observed data using pyro.sample with the obs keyword argument
    for i in range(len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

For this model the observations are conditionally independent given the latent random variable **latent_fairness**. To explicitly mark this in Pyro we basically just need to replace the Python builtin range with the Pyro construct plate: